In [7]:
from citipy import citipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from datetime import datetime
from config2 import weather_api_key

In [8]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
longs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_longs = zip(lats, longs)

In [9]:
# Create a list to hold cities
cities = []

# identify each city nearest to each set of coordinates
for coordinates in lats_longs:
    city = citipy.nearest_city(coordinates[0], coordinates[1]).city_name
    
    if city not in cities:
        cities.append(city)
len(cities)

771

In [ ]:
# 1. Create empty city list to hold cities
city_data = []

# print the start of logging
print("Beginning Data Retrieval")
print("------------------------")

# create counters for looping through cities
record_count = 1
set_count = 1

# 2. loop through cities
for i, city in enumerate(cities):
    # 3. group cities in sets of 50
    if (i % 50 == 0 and i >=50):
        set_count += 1
        record_count += 1
        time.sleep(60)

    # 4. build city url endpoint for each city
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # 5. log the url and record/set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # add 1 to record count
    record_count += 1
    
    # 6. Run an API request for each of the cities.
    try:
        # 6. Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # 7. Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # 8. Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# convert the array of JSON dict into dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

In [ ]:
# reorder columns
city_data2_df = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data2_df = city_data_df[city_data2_df]

In [ ]:
city_data2_df.head(10)

In [ ]:
# create file output and save city_data2_df in the csv
save_file = "Weather_Database/WeatherPy_DataBase.csv"
city_data2_df.to_csv(save_file, index_label="City ID")